In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

load_dotenv()

model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.5
)


#### **Static Prompt**

In [2]:
response = model.invoke("Welche sprachen sprichst du?")
print(response.content)

Ich wurde von Google trainiert und spreche viele Sprachen. Ich kann Texte auf Deutsch verstehen und generieren.  Ich kann auch in anderen Sprachen kommunizieren, aber meine Fähigkeiten variieren je nach Sprache.  Meine Stärke liegt in der Verarbeitung und Generierung von Texten, nicht im direkten Sprechen.


#### **Dynamic Prompt Template**

In [4]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template="""
    You are a {profession}, who only talks about related subject.
    If someone asks you a question that is not related to your profession, you will answer with "I don't know".
    
    Answer the following question:
    {question}
    """,
    input_variables=["profession", "question"]
)

prompt_template = prompt.invoke(
    {
        "profession": "doctor",
        "question": "What is the capital of France?"
    }
)

response = model.invoke(prompt_template)
print(response.content)

I don't know.


##### **Save and Load Template**

In [5]:
prompt.save('template.json')

In [6]:
from langchain_core.prompts import load_prompt

template = load_prompt('template.json')

#### **Chat Prompt Template**

In [ ]:
# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

